In [32]:
pip install pandas numpy openpyxl

Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

In [34]:
np.random.seed(42)

In [35]:
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
months = pd.date_range(start=start_date, end=end_date, freq='MS')


In [36]:
accounts = {
    'Revenue': {
        'Product Sales': {'base': 150000, 'variance': 0.15, 'trend': 1.02},
        'Service Revenue': {'base': 60000, 'variance': 0.12, 'trend': 1.015},
        'Subscription Revenue': {'base': 40000, 'variance': 0.08, 'trend': 1.03}
    },
    'COGS': {
        'Materials': {'base': -45000, 'variance': 0.10, 'trend': 1.01},
        'Direct Labor': {'base': -35000, 'variance': 0.08, 'trend': 1.02},
        'Manufacturing Overhead': {'base': -20000, 'variance': 0.12, 'trend': 1.01}
    },
    'OpEx': {
        'Salaries & Wages': {'base': -95000, 'variance': 0.05, 'trend': 1.025},
        'Rent': {'base': -18000, 'variance': 0.02, 'trend': 1.00},
        'Marketing': {'base': -15000, 'variance': 0.25, 'trend': 1.04},
        'Utilities': {'base': -5000, 'variance': 0.15, 'trend': 1.02},
        'Insurance': {'base': -8000, 'variance': 0.05, 'trend': 1.01},
        'Office Supplies': {'base': -3000, 'variance': 0.20, 'trend': 1.00},
        'IT & Software': {'base': -7000, 'variance': 0.10, 'trend': 1.05}
    }
}

departments = {
    'Product Sales': 'Sales',
    'Service Revenue': 'Services',
    'Subscription Revenue': 'Sales',
    'Materials': 'Production',
    'Direct Labor': 'Production',
    'Manufacturing Overhead': 'Production',
    'Salaries & Wages': 'HR',
    'Rent': 'Admin',
    'Marketing': 'Marketing',
    'Utilities': 'Admin',
    'Insurance': 'Admin',
    'Office Supplies': 'Admin',
    'IT & Software': 'IT'
}

In [37]:
# Generate transactions
transactions = []

for i, month in enumerate(months):
    month_multiplier = 1 + (i * 0.01)  # Overall growth trend
    
    # Add seasonal variation (Q4 boost)
    if month.month in [11, 12]:
        seasonal = 1.2
    elif month.month in [1, 2]:
        seasonal = 0.9
    else:
        seasonal = 1.0

In [38]:
 for category, accounts_dict in accounts.items():
        for account_name, params in accounts_dict.items():
            # Calculate actual amount
            base = params['base']
            variance = params['variance']
            trend = params['trend'] ** i
            
            actual = base * trend * seasonal * (1 + np.random.uniform(-variance, variance))
            
            # Calculate budget (slightly different from actual)
            budget = base * trend * (1 + np.random.uniform(-0.05, 0.05))
            
            # Create transaction
            transactions.append({
                'Date': month,
                'Account_Category': category,
                'Account_Name': account_name,
                'Department': departments[account_name],
                'Amount_Actual': round(actual, 2),
                'Amount_Budget': round(budget, 2),
                'Variance': round(actual - budget, 2),
                'Variance_Percent': round(((actual - budget) / abs(budget)) * 100, 2) if budget != 0 else 0
            })

In [39]:
# Create DataFrame
df = pd.DataFrame(transactions)

# Add month and year columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month_Name'] = df['Date'].dt.strftime('%B')
df['Year_Month'] = df['Date'].dt.strftime('%Y-%m')
df['Quarter'] = 'Q' + df['Date'].dt.quarter.astype(str)

# Add Account Type
df['Account_Type'] = df['Account_Category'].apply(
    lambda x: 'Income' if x == 'Revenue' else 'Expense'
)


In [40]:
# Save to Excel
with pd.ExcelWriter('Financial_Data.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Transactions', index=False)
    
    # Chart of Accounts
    accounts_list = []
    for category, accounts_dict in accounts.items():
        for account_name in accounts_dict.keys():
            accounts_list.append({
                'Account_Name': account_name,
                'Account_Category': category,
                'Account_Type': 'Income' if category == 'Revenue' else 'Expense',
                'Department': departments[account_name]
            })
    
    pd.DataFrame(accounts_list).to_excel(writer, sheet_name='Chart_of_Accounts', index=False)

print("✅ Financial data generated successfully!")
print(f"📊 Total records: {len(df)}")
print(f"📅 Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"💰 Total Revenue: ${df[df['Account_Type']=='Income']['Amount_Actual'].sum():,.2f}")
print(f"💸 Total Expenses: ${df[df['Account_Type']=='Expense']['Amount_Actual'].sum():,.2f}")
print(f"💵 Net Profit: ${df['Amount_Actual'].sum():,.2f}")

✅ Financial data generated successfully!
📊 Total records: 13
📅 Date range: 2024-12-01 00:00:00 to 2024-12-01 00:00:00
💰 Total Revenue: $469,726.16
💸 Total Expenses: $-475,435.95
💵 Net Profit: $-5,709.79
